<a href="https://colab.research.google.com/github/felix-thiago/EP2_Ciencia_de_dados/blob/main/EP2_Ciencia_de_Dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [97]:
import os
!pip install neo4j
from neo4j import GraphDatabase
import json
import requests
import re


senha = "w3NXsLlWKIny0lJ_DCVGCBrB7B-YvZQEbsr_PeM59Jk"
url = "neo4j+s://dd7a4afa.databases.neo4j.io"
usuario = "neo4j"

In [112]:
class PokemonGraph:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def clear_database(self):
        query = "MATCH (n) DETACH DELETE n"
        with self.driver.session() as session:
            session.run(query)

    @staticmethod
    def _clear_data(tx):
        # Apagar todos os Pokémon, tipos, habilidades e evoluções
        tx.run("MATCH (n) DETACH DELETE n")
        print("Banco de dados limpo!")

    def create_pokemon(self, pokemon_id, name, weight, height):
        with self.driver.session() as session:
            session.execute_write(self._create_pokemon, pokemon_id, name, weight, height)

    @staticmethod
    def _create_pokemon(tx, pokemon_id, name, weight, height):
        tx.run(
            "CREATE (p:Pokemon {id: $pokemon_id, name: $name, weight: $weight, height: $height})",
            pokemon_id=pokemon_id, name=name, weight=weight, height=height
        )

    def create_type(self, name):
        with self.driver.session() as session:
            session.execute_write(self._create_type, name)

    @staticmethod
    def _create_type(tx, name):
        tx.run("CREATE (t:Type {name: $name})", name=name)

    def create_relation(self, pokemon_name, type_name):
        with self.driver.session() as session:
            session.execute_write(self._create_relation, pokemon_name, type_name)

    @staticmethod
    def _create_relation(tx, pokemon_name, type_name):
        tx.run("""
            MATCH (p:Pokemon {name: $pokemon_name})
            MATCH (t:Type {name: $type_name})
            CREATE (p)-[:HAS_TYPE]->(t)
            """, pokemon_name=pokemon_name, type_name=type_name)

    def create_evolution_relation(self, pokemon_name, evolution_name):
        with self.driver.session() as session:
            session.execute_write(self._create_evolution_relation, pokemon_name, evolution_name)

    @staticmethod
    def _create_evolution_relation(tx, pokemon_name, evolution_name):
        tx.run("""
            MATCH (p:Pokemon {name: $pokemon_name})
            MATCH (e:Pokemon {name: $evolution_name})
            CREATE (p)-[:EVOLVES_INTO]->(e)
            """, pokemon_name=pokemon_name, evolution_name=evolution_name)

    def create_ability(self, ability_name, ability_url):
        with self.driver.session() as session:
            session.execute_write(self._create_ability, ability_name, ability_url)

    @staticmethod
    def _create_ability(tx, ability_name, ability_url):
        tx.run("CREATE (a:Ability {name: $ability_name, url: $ability_url})",
               ability_name=ability_name, ability_url=ability_url)

    def create_ability_relation(self, pokemon_name, ability_name):
        with self.driver.session() as session:
            session.execute_write(self._create_ability_relation, pokemon_name, ability_name)

    @staticmethod
    def _create_ability_relation(tx, pokemon_name, ability_name):
        tx.run("""
            MATCH (p:Pokemon {name: $pokemon_name})
            MATCH (a:Ability {name: $ability_name})
            CREATE (p)-[:HAS_ABILITY]->(a)
            """, pokemon_name=pokemon_name, ability_name=ability_name)

# Agora vamos usar o código de inserção com os dados
uri = "neo4j+s://dd7a4afa.databases.neo4j.io"  # Substitua pela URL do seu banco
user = "neo4j"
password = "w3NXsLlWKIny0lJ_DCVGCBrB7B-YvZQEbsr_PeM59Jk"  # Sua senha do Neo4j

graph = PokemonGraph(uri, user, password)

# Limpar o banco de dados antes de inserir novos dados
graph.clear_database()

# Baixar o JSON
url = 'https://raw.githubusercontent.com/felix-thiago/EP01_Ciencia-de-Dados_Senac/refs/heads/main/pokemons.json'
headers = {'Accept': 'application/json'}
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    print("Arquivo JSON carregado com sucesso!")
else:
    print(f"Erro ao acessar a URL: {response.status_code}")

def normalize_id(pokemon_id):
    # Converte o ID para string, remove caracteres não numéricos e converte de volta para inteiro
    if isinstance(pokemon_id, str):
        return str(int(re.sub(r'[^\d]', '', pokemon_id)))
    return pokemon_id  # Se já for numérico, retorna como está

# Normalizar os IDs de cada Pokémon
for pokemon in data:
    pokemon['pokemon_id'] = normalize_id(pokemon['pokemon_id'])

# Exibir os dados com IDs normalizados
for pokemon in data:
    print(f"ID: {pokemon['pokemon_id']}, Name: {pokemon['pokemon_name']}, Weight: {pokemon['pokemon_weight']}, Height: {pokemon['pokemon_height']}")



#____________________________________________________________________________________________________________________

# Função para verificar se o Pokémon já existe
def pokemon_exists(pokemon_name):
    query = f'MATCH (p:Pokemon {{name: "{pokemon_name}"}}) RETURN p LIMIT 1'
    with graph.driver.session() as session:
        # Usar a transação de leitura para buscar o Pokémon
        result = session.read_transaction(lambda tx: tx.run(query).data())
        return bool(result)  # Retorna True se encontrar o Pokémon


def create_pokemon(self, pokemon_id, name, weight, height):
    query = (
        "MERGE (p:Pokemon {id: $pokemon_id}) "
        "ON CREATE SET p.name = $name, p.weight = $weight, p.height = $height"
    )
    with self.driver.session() as session:
        session.run(query, pokemon_id=pokemon_id, name=name, weight=weight, height=height)


# Formatar o ID como uma string sem zeros à esquerda
pokemon_id = str(int(pokemon['pokemon_id']))


# Inserir os dados no Neo4j
for pokemon in data:
    pokemon_id = str(int(pokemon['pokemon_id'])).lstrip('#')  # Remover o '#' do ID
    pokemon_name = pokemon['pokemon_name']
    pokemon_weight = pokemon["pokemon_weight"]
    pokemon_height = pokemon["pokemon_height"]
    next_evolutions = pokemon["next_evolutions"]
    pokemon_types = pokemon["pokemon_types"]
    pokemon_abilities = pokemon.get("abilities", [])  # Usar .get() para evitar KeyError

    # Verificar se o Pokémon já existe, se não, criar o nó
    if not pokemon_exists(pokemon_name):
        graph.create_pokemon(pokemon_id, pokemon_name, pokemon_weight, pokemon_height)

    # Criar tipos e relações
    for poke_type in pokemon['pokemon_types']:
        graph.create_type(poke_type)
        graph.create_relation(pokemon_name, poke_type)

    # Criar relações de evolução (se aplicável)
    for evolution in pokemon['next_evolutions']:
        evolution_name = evolution['evo_name']
        evolution_id = str(evolution['evo_num']).lstrip('#')  # Remover o '#' do ID da evolução
"""
        # Verificar se o Pokémon de evolução já existe, se não, criar o nó (mas com valores fictícios)
        if not pokemon_exists(evolution_name):
            graph.create_pokemon(evolution_id, evolution_name, 0, 0)  # Dados de evolução, com valores fictícios
        graph.create_evolution_relation(pokemon_name, evolution_name)
"""
# Criar habilidades e relações com o Pokémon
for ability in pokemon_abilities:
    graph.create_ability(ability)  # Criar o nó para habilidade
    graph.create_ability_relation(pokemon_name, ability)  # Criar a relação entre Pokémon e habilidade

# Fechar a conexão após o uso
graph.close()


Arquivo JSON carregado com sucesso!
ID: 1, Name: Bulbasaur, Weight: 6.9 kg, Height: 0.7 m
ID: 2, Name: Ivysaur, Weight: 13.0 kg, Height: 1.0 m
ID: 3, Name: Venusaur, Weight: 100.0 kg, Height: 2.0 m
ID: 4, Name: Charmander, Weight: 8.5 kg, Height: 0.6 m
ID: 5, Name: Charmeleon, Weight: 19.0 kg, Height: 1.1 m
ID: 6, Name: Charizard, Weight: 90.5 kg, Height: 1.7 m
ID: 7, Name: Squirtle, Weight: 9.0 kg, Height: 0.5 m
ID: 8, Name: Wartortle, Weight: 22.5 kg, Height: 1.0 m
ID: 9, Name: Blastoise, Weight: 85.5 kg, Height: 1.6 m
ID: 10, Name: Caterpie, Weight: 2.9 kg, Height: 0.3 m
ID: 11, Name: Metapod, Weight: 9.9 kg, Height: 0.7 m
ID: 12, Name: Butterfree, Weight: 32.0 kg, Height: 1.1 m
ID: 13, Name: Weedle, Weight: 3.2 kg, Height: 0.3 m
ID: 14, Name: Kakuna, Weight: 10.0 kg, Height: 0.6 m
ID: 15, Name: Beedrill, Weight: 29.5 kg, Height: 1.0 m
ID: 16, Name: Pidgey, Weight: 1.8 kg, Height: 0.3 m
ID: 17, Name: Pidgeotto, Weight: 30.0 kg, Height: 1.1 m
ID: 18, Name: Pidgeot, Weight: 39.5 kg, H

<ipython-input-112-e3543c495323>:127: DeprecationWarning: read_transaction has been renamed to execute_read
  result = session.read_transaction(lambda tx: tx.run(query).data())


# Consultas

In [102]:
def pokemons_atacam_pikachu(graph):
    query = """
    MATCH (p:Pokemon {name: "Pikachu"})-[:HAS_TYPE]->(p_type:Type)
    MATCH (attacker:Pokemon)-[:HAS_TYPE]->(attacker_type:Type)
    WHERE p_type.name = "Electric"
      AND attacker_type.name = "Ground"  // Supondo que Ground é o tipo que causa mais dano ao Pikachu
      AND attacker.weight > 10  // Pokémon com peso superior a 10kg
    RETURN attacker.name AS Attacker, attacker.weight AS Weight
    """

    with graph.driver.session() as session:
        result = session.run(query)
        return [(record["Attacker"], record["Weight"]) for record in result]

# Chamada da função
attacking_pokemons = pokemons_atacam_pikachu(graph)
print(attacking_pokemons)


<ipython-input-102-4668fd496989>:11: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with graph.driver.session() as session:


[]


In [103]:
def tipo_comum_atacado_por_gelo(graph):
    query = """
    MATCH (attacker:Pokemon)-[:HAS_TYPE]->(attacker_type:Type)
    MATCH (victim:Pokemon)-[:HAS_TYPE]->(victim_type:Type)
    WHERE attacker_type.name = "Ice"
    RETURN victim_type.name AS Type, COUNT(victim_type) AS Frequency
    ORDER BY Frequency DESC
    LIMIT 1
    """

    with graph.driver.session() as session:
        result = session.run(query)
        return result.single()

# Chamada da função
most_common_type = tipo_comum_atacado_por_gelo(graph)
print(most_common_type)

<ipython-input-103-7274782ecdc6>:11: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with graph.driver.session() as session:


<Record Type='Water' Frequency=33410520>


In [113]:
def evolucoes_que_dobram_peso(graph):
    query = """
    MATCH (p:Pokemon)-[:EVOLVES_INTO]->(evo1:Pokemon)-[:EVOLVES_INTO]->(evo2:Pokemon)
    WHERE evo1.weight >= p.weight * 2  // Verifica se a primeira evolução dobrou o peso
      OR evo2.weight >= evo1.weight * 2  // Verifica se a segunda evolução dobrou o peso
    RETURN COUNT(p) AS EvolutionCount
    """

    with graph.driver.session() as session:
        result = session.run(query)
        return result.single()

# Chamada da função
evolution_count = evolucoes_que_dobram_peso(graph)
print(evolution_count)


<ipython-input-113-2bce612afbb1>:9: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with graph.driver.session() as session:


<Record EvolutionCount=0>
